# Research Topic Assistant

This notebook takes a research topic, generates sub-questions using an LLM, retrieves answers, and summarizes the results into a final paragraph.


In [1]:
!pip install openai

In [2]:
import os
from openai import OpenAI
import json
from typing import List, Dict


In [4]:
import os
from openai import OpenAI
import json
from typing import List, Dict
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

## Step 1: Generate Sub-Questions from Research Topic


In [5]:
def generate_sub_questions(topic: str, num_questions: int = 3) -> List[str]:
    """
    Uses an LLM to generate sub-questions related to the research topic.
    """
    prompt = f"""Given the research topic: "{topic}"

Generate exactly {num_questions} specific sub-questions that would help someone understand this topic better.
The questions should be diverse and cover different aspects of the topic.

Return ONLY the questions, one per line, numbered 1-{num_questions}."""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful research assistant that generates insightful questions."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=300
    )

    # Parse the response to extract questions
    questions_text = response.choices[0].message.content.strip()
    questions = []

    for line in questions_text.split('\n'):
        line = line.strip()
        if line:
            question = line.split('. ', 1)[-1] if '. ' in line else line
            question = question.split(') ', 1)[-1] if ') ' in line else question
            questions.append(question)

    return questions[:num_questions]


## Step 2: Retrieve Answers for Sub-Questions


In [6]:
def get_answer(question: str, topic: str) -> str:
    """
    Uses an LLM to generate a short response to a sub-question.
    """
    prompt = f"""Context: We are researching "{topic}"

Question: {question}

Provide a concise answer in 2-3 sentences."""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a knowledgeable research assistant. Provide clear, concise answers."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=150
    )

    return response.choices[0].message.content.strip()


def retrieve_answers(questions: List[str], topic: str) -> List[Dict[str, str]]:
    """
    Retrieves answers for all sub-questions.
    """
    qa_pairs = []

    for question in questions:
        answer = get_answer(question, topic)
        qa_pairs.append({
            'question': question,
            'answer': answer
        })

    return qa_pairs


## Step 3: Summarize Results into Final Paragraph


In [7]:
def summarize_results(topic: str, qa_pairs: List[Dict[str, str]]) -> str:
    """
    Synthesizes all Q&A pairs into a coherent final paragraph.
    """
    # Format the Q&A pairs for the prompt
    qa_text = "\n\n".join([
        f"Q: {pair['question']}\nA: {pair['answer']}"
        for pair in qa_pairs
    ])

    prompt = f"""Research Topic: "{topic}"

Here are answers to key questions about this topic:

{qa_text}

Please write a single comprehensive paragraph (4-6 sentences) that synthesizes all the information above into a coherent overview of the topic. The paragraph should flow naturally and provide a complete understanding of the research topic."""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert at synthesizing information into clear, coherent summaries."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=300
    )

    return response.choices[0].message.content.strip()


## Research Assistant Function


In [8]:
def research_assistant(topic: str) -> Dict:
    """
    Complete research assistant pipeline.

    Args:
        topic: The research topic to explore

    Returns:
        Dictionary containing questions, answers, and final summary
    """
    print(f"Researching topic: {topic}\n")
    print("=" * 80)

    print("\nStep 1: Generating sub-questions...\n")
    questions = generate_sub_questions(topic, num_questions=3)

    for i, q in enumerate(questions, 1):
        print(f"  {i}. {q}")

    print("\nStep 2: Retrieving answers...\n")
    qa_pairs = retrieve_answers(questions, topic)

    for i, pair in enumerate(qa_pairs, 1):
        print(f"  Q{i}: {pair['question']}")
        print(f"  A{i}: {pair['answer']}\n")

    print("Step 3: Generating final summary...\n")
    summary = summarize_results(topic, qa_pairs)

    print("=" * 80)
    print("\nFINAL SUMMARY:\n")
    print(summary)
    print("\n" + "=" * 80)

    return {
        'topic': topic,
        'questions': questions,
        'qa_pairs': qa_pairs,
        'summary': summary
    }


## Run the Research Assistant


In [9]:
# INPUT: Enter your research topic here
research_topic = "AI vs Human"

results = research_assistant(research_topic)


Researching topic: AI vs Human


Step 1: Generating sub-questions...

  1. What are the specific areas where AI has surpassed human performance and what implications does this have for the future?
  2. How does the process of human decision-making differ from that of AI, especially in terms of ethics and unpredictability?
  3. How does the proliferation of AI in various sectors affect human job security and what are the potential solutions to this problem?

Step 2: Retrieving answers...

  Q1: What are the specific areas where AI has surpassed human performance and what implications does this have for the future?
  A1: AI has surpassed human performance in areas such as data analysis, image recognition, and certain games like chess and Go, given its ability to process large amounts of data quickly and accurately. This suggests a future where tasks requiring detailed pattern recognition and data processing could be automated, potentially leading to increased efficiency but also job disp